#### <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:180%; text-align:left;padding:3.0px; background: maroon; border-bottom: 8px solid black" > TABLE OF CONTENTS<br><div>
* [IMPORTS](#1)
* [Introduction](#2)
* [GraphQA Chain](#3)
* [Custom Chain](#4)
* [Semantic Retrieval](#5)
* [Adding Memory](#6)
* [Final Chain](#7)
* [Next Steps](#8)


In [1]:
import os

import langchain
## Chains
from operator import itemgetter

from langchain_community.graphs import Neo4jGraph

from langchain_core.runnables import RunnableLambda, RunnablePassthrough

from langchain_community.chains.graph_qa.prompts import (
    CYPHER_QA_PROMPT,
    CYPHER_GENERATION_PROMPT
)

# Prompts:
from langchain_core.prompts import (
    PromptTemplate
)
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import HumanMessage


## LLMs:
from langchain_openai import OpenAI, ChatOpenAI

from langchain_community.vectorstores import Neo4jVector
from langchain.embeddings import HuggingFaceEmbeddings

# Memory
## Memory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

langchain.debug=False

<a id="2"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color: white; font-size:120%; text-align:left;padding:3.0px; background: maroon; border-bottom: 8px solid black" > Introduction<br><div>

In this notebook we are going to show how to create a 'Custom' GraphRAG set up, using as an example the GraphQAChain client provided by Langchain.

<a id="3"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color: white; font-size:120%; text-align:left;padding:3.0px; background: maroon; border-bottom: 8px solid black" > GraphQA Chain<br><div>

We have based this development in the [GraphCypherQAChain](https://api.python.langchain.com/en/latest/chains/langchain_community.chains.graph_qa.cypher.GraphCypherQAChain.html) chain. We are going to show how to replicate it's main behaviour here, but adapting it to the LCEL langchain notation, considering that those old chains are deprecated. In this way we ensure that our final solution will be more 'production ready', and also will be more customized.


Inspecting the chain class definition we realized that, by default, it uses two predefined prompts:

* Query generation prompt: Handles the conversion from a user query to a CYPHER query

* QuestionAnswer prompt: Once the context has been retrieved from our Knowledge graph, this prompt handles the conversation

Let's take a look to this prompts:

## Visualize the prompts

In [2]:
CYPHER_GENERATION_PROMPT

PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}')

In [3]:
CYPHER_GENERATION_PROMPT.invoke({'question':"How to build a confusion matrix with plotly?",'schema':'This will be the schema of the graph'}).text

'Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\nThis will be the schema of the graph\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\nHow to build a confusion matrix with plotly?'

In [4]:
CYPHER_QA_PROMPT

PromptTemplate(input_variables=['context', 'question'], template="You are an assistant that helps to form nice and human understandable answers.\nThe information part contains the provided information that you must use to construct an answer.\nThe provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.\nMake the answer sound as a response to the question. Do not mention that you based the result on the given information.\nHere is an example:\n\nQuestion: Which managers own Neo4j stocks?\nContext:[manager:CTL LLC, manager:JANE STREET GROUP LLC]\nHelpful Answer: CTL LLC, JANE STREET GROUP LLC owns Neo4j stocks.\n\nFollow this example when generating answers.\nIf the provided information is empty, say that you don't know the answer.\nInformation:\n{context}\n\nQuestion: {question}\nHelpful Answer:")

As we can see, this second prompt is just to handle the conversation, once the query has retrieved some content, so we will focus in the first one.

<a id="4"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color: white; font-size:120%; text-align:left;padding:3.0px; background: maroon; border-bottom: 8px solid black" > Custom Chain<br><div>

We are going to replicate here a chain that has mainly the same behaviour, but adapted to our use case.

In [5]:
llm= ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo") # gpt-4-0125-preview occasionally has issues

graph_cypher_chain = CYPHER_GENERATION_PROMPT | llm

In [6]:
result = graph_cypher_chain.invoke({'question':"How to build a confusion matrix with plotly?",'schema':'This will be the schema of the graph'})

In [7]:
result.content

'MATCH (a:Actual)-[r:PREDICTED_AS]->(p:Predicted)\nWITH {label: a.label, prediction: p.label} as data, count(*) as count\nRETURN data, count'

As we can see this value makes not sense at all, because we have not provided the schema to the LLM yet, let's reproduce this part based in the reference Chain

### Load a graph from neo4j

We do this with the wrapper that langchain community offers. We could create our own, but for now we will just stick to it.
The main advantage of this client is that directly create an schema for us, so we can contextualize the LLM.

In [8]:
graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password=os.environ['NEO4J_PASSWORD'],database='graphrag')

### This function allows us to directly get the schema from the database

In [9]:
graph.get_schema

'Node properties:\nFunction {description: STRING, embedding: LIST, name: STRING, code: STRING, file_path: STRING}\nArea {name: STRING}\nSubArea {name: STRING}\nFramework {name: STRING}\nClass {description: STRING, name: STRING, code: STRING, file_path: STRING}\nRelationship properties:\n\nThe relationships:\n(:Area)-[:CONTAINS_SUBAREA]->(:SubArea)\n(:Area)-[:CONTAINS_FRAMEWORK]->(:Framework)\n(:SubArea)-[:CONTAINS_FRAMEWORK]->(:Framework)\n(:Framework)-[:CONTAINS_FUNCTION]->(:Function)\n(:Framework)-[:CONTAINS_CLASS]->(:Class)'

In [10]:
graph.get_structured_schema

{'node_props': {'Function': [{'property': 'description', 'type': 'STRING'},
   {'property': 'embedding', 'type': 'LIST'},
   {'property': 'name', 'type': 'STRING'},
   {'property': 'code', 'type': 'STRING'},
   {'property': 'file_path', 'type': 'STRING'}],
  'Area': [{'property': 'name', 'type': 'STRING'}],
  'SubArea': [{'property': 'name', 'type': 'STRING'}],
  'Framework': [{'property': 'name', 'type': 'STRING'}],
  'Class': [{'property': 'description', 'type': 'STRING'},
   {'property': 'name', 'type': 'STRING'},
   {'property': 'code', 'type': 'STRING'},
   {'property': 'file_path', 'type': 'STRING'}]},
 'rel_props': {},
 'relationships': [{'start': 'Area',
   'type': 'CONTAINS_SUBAREA',
   'end': 'SubArea'},
  {'start': 'Area', 'type': 'CONTAINS_FRAMEWORK', 'end': 'Framework'},
  {'start': 'SubArea', 'type': 'CONTAINS_FRAMEWORK', 'end': 'Framework'},
  {'start': 'Framework', 'type': 'CONTAINS_FUNCTION', 'end': 'Function'},
  {'start': 'Framework', 'type': 'CONTAINS_CLASS', 'end':

## Invoke the function with this schema as input

In [11]:
result = graph_cypher_chain.invoke({'question':"How to use plotly framework?",'schema':graph.get_schema})

In [12]:
result.content

'MATCH (a:Area)-[:CONTAINS_FRAMEWORK]->(f:Framework {name: "plotly"})\nRETURN f'

Other advantage of the langchain graph client is that allows to directly run the queries returned by this first LLM:

In [13]:
context = graph.query(result.content)[:5]

In [14]:
context

[{'f': {'name': 'plotly'}}]

### Here we added the keyword 'framework' in the question, but that is hightly unlikely in a normal query

In [15]:
result = graph_cypher_chain.invoke({'question':"How to use plotly?",'schema':graph.get_schema})

result.content

"MATCH (a:Area)-[:CONTAINS_SUBAREA]->(sa:SubArea)-[:CONTAINS_FRAMEWORK]->(f:Framework)-[:CONTAINS_FUNCTION]->(func:Function)\nWHERE func.name = 'plotly'\nRETURN a, sa, f, func;"

In [16]:
context = graph.query(result.content)[:5]
context

[]

The first problem that we see is that considering how our graph is built, the entities are difficult to assign only by their name. This is a generic problem of this kind of solution. Entities should have a very descriptive name (Person, Organization...) so they can be eassily identified by a general LLM. So we should try and contextualize better about the entities that the LLM can expect. For that we will take as reference the base prompt used in the GraphCypherChain and add the following lines defining the entities for our problem.

You are a helpful assistant that understands the context of data science and can generate Cypher queries to retrieve information from a Neo4j database.

The database schema includes the following entities:
- Data Preprocessing Area: Nodes labeled as 'DataPreprocessingArea' representing areas of data preprocessing.
- SubArea: Nodes labeled as 'SubArea' representing sub-areas within data preprocessing.
- Framework: Nodes labeled as 'Framework' representing frameworks used in data science.
- Class: Nodes labeled as 'Class' representing a set of functions defining a Python class within a framework.
- Function: Nodes labeled as 'Function' representing specific functions within frameworks.

In [17]:


cypher_gen_prompt = PromptTemplate.from_template(
    """
    You are a Cypher language expert.
    Your Task:Generate Cypher statement to query a graph database.
    To better contextualize, the Graph database is mapping the Data Science implementations using python
    and is divided in the following entities:

    Instructions:
    Use only the provided relationship types and properties in the schema.
    Do not use any other relationship types or properties that are not provided.
    Schema:
    {schema}
    Note: Do not include any explanations or apologies in your responses.
    Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
    Do not include any text except the generated Cypher statement.
        - Data Preprocessing Area: Nodes labeled as 'Area' representing areas of Data Science, like 'Data Visualization' or 'Data Preprocessing'.
        - SubArea: Nodes labeled as 'SubArea' representing sub-areas within data preprocessing. This field is optional, some of the nodes may not have a relation with a 'SubArea' node, so generally 
        should not be added in the query.
        - Framework: Nodes labeled as 'Framework' representing frameworks used in data science.
        - Class: Nodes labeled as 'Class' representing a set of functions defining a Python class within a framework.
        - Function: Nodes labeled as 'Function' representing custom functions built on top of those frameworks.
    Nodes do not neccesarily have parents of each type of label.

    Your main focus should be to identify the Framework and the Function that is being asked.
    The question is:
    {question}
    """
   
)

In [18]:
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo") # gpt-4-0125-preview occasionally has issues

custom_prompt_chain = cypher_gen_prompt | llm

In [19]:
result = custom_prompt_chain.invoke({'question':"How to use plotly?",'schema':graph.get_schema})

In [20]:
result.content

"MATCH (:Framework{name: 'plotly'})-[:CONTAINS_FUNCTION]->(f:Function)\nRETURN f;"

In [21]:

result = custom_prompt_chain.invoke({'question':"How to use plotly to generate a confusion matrix?",'schema':graph.get_schema})
result.content

'MATCH (:Framework{name: "plotly"})-[:CONTAINS_FUNCTION]->(:Function{name: "generate_confusion_matrix"})\nRETURN *;'

In [22]:
context = graph.query(result.content)[:5]
context

ValueError: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: RETURN * is not allowed when there are no variables in scope (line 2, column 1 (offset: 104))
"RETURN *;"
 ^}

In [ ]:

result = custom_prompt_chain.invoke({'question':"How to use plotly to generate a 'plot_confusion_matrix' function? Provide the code",'schema':graph.get_schema})
result

AIMessage(content="MATCH (a:Area)-[:CONTAINS_FRAMEWORK]->(f:Framework)-[:CONTAINS_FUNCTION]->(func:Function)\nWHERE f.name = 'plotly' AND func.name = 'plot_confusion_matrix'\nRETURN func.code", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 477, 'total_tokens': 525}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-06f7b36a-04ba-4b7c-a7da-27ba12d48e27-0', usage_metadata={'input_tokens': 477, 'output_tokens': 48, 'total_tokens': 525})

In [ ]:
context = graph.query(result.content)[:5]
context

[{'func.code': 'def plot_confusion_matrix(confusion_matrix, class_names):     """     Generates a confusion matrix plot from a sklearn confusion matrix object.      Parameters:     - confusion_matrix (array): Numpy array containing the confusion matrix information.     - class_names (list of str): List of class names corresponding to the labels.      Returns:     - fig (plotly.graph_objects.Figure): The Plotly figure object for the confusion matrix plot.     """     fig = ff.create_annotated_heatmap(         z=confusion_matrix,         x=class_names,         y=class_names,         colorscale=\'Blues\',         showscale=True     )     fig.update_layout(title=\'Confusion Matrix\', xaxis_title=\'Predicted Label\', yaxis_title=\'True Label\')     fig.update_traces(text=confusion_matrix.astype(str), texttemplate=\'%{text}\')     return fig'}]

### We can see that after contextualizing what can be understood as a 'Framework' in our graph, the LLM is correctly identifying plotly as a framework

In [ ]:
qa_prompt_template =  PromptTemplate.from_template(
    """You are an assistant that helps to form nice and human understandable answers.
    The information part contains the provided information that you must use to construct an answer.
    The provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
    Make the answer sound as a response to the question. Do not mention that you based the result on the given information.
    If the provided information is empty, say that you don't know the answer.
    Information:
    {context}
    
    Question: {question}
    Helpful Answer:
    """
)

We have sligthly adapted the prompt from the reference chain to our end, removing the example mainly. Let's build the complete chain

In [ ]:
def run_cypher_query(query):
    try:
        print("Generated query---->",query.content)
        node_contents = graph.query(query.content)[:5]
        return node_contents
    except: 
        return ""

In [ ]:
full_qa_chain = {'context': cypher_gen_prompt | llm | RunnableLambda(run_cypher_query), 'question': RunnablePassthrough()} | qa_prompt_template | llm

langchain.debug=True

full_qa_chain.invoke({'question':"How to use plotly to generate a 'plot_confusion_matrix' function? Provide the code",'schema':graph.get_schema})

NameError: name 'qa_prompt_template' is not defined

This was everything regarding the usage of the Graph based in a query retrieval strategy. But seing that this is not always accurate and may not give any result, we are going to mix it with a 'Semantic simmilarity' retrieval procedure.

<a id="5"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color: white; font-size:120%; text-align:left;padding:3.0px; background: maroon; border-bottom: 8px solid black" > Semantic retrieval<br><div>

In this section we are going to define again an instance of the vector index we created in the [Neo4j Vector Store Tutorial](https://github.com/hectorrrr/langchain_utils/blob/2dd4cdf88dd6c11d5f882db6490e302bf2bdf961/examples/neo4j_vector_store.ipynb). Then we will use it as a retriever and show how we can create a simple chain usign this information.

In [ ]:
database = "graphrag"  # default index name
## Uncomment de wanted embedding model
# embeddings = OpenAIEmbeddings()
model_name = "sentence-transformers/all-MiniLM-L6-v2" # You can specify any sentence-transformer model from the hub
embeddings = HuggingFaceEmbeddings(model_name=model_name)


# The vector index name was assigned by default
store = Neo4jVector.from_existing_index(
    embeddings,
    url=os.environ["NEO4J_URL"],
    username=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"],
    index_name="vector",
    search_type="hybrid",
    keyword_index_name="keyword",
    database=database
)



c:\Users\hecto\OneDrive\Escritorio\github_projects\langchain_utils\langchain_env\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\hecto\OneDrive\Escritorio\github_projects\langchain_utils\langchain_env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\hecto\OneDrive\Escritorio\github_projects\langchain_utils\langchain_env\lib\site-packages\transformers\tokenization_utils_base.py:1601: Fu

In [ ]:
retriever = store.as_retriever(search_kwargs= {'k':2, 'score_threshold':0.7}) #, score_threshold=0.7

In [ ]:
retriever.invoke("How to generate a confusion matrix with plotly?")

[Document(metadata={'file_path': 'visualization\\plotly\\machine_learning_evaluation_plots.py', 'name': 'plot_confusion_matrix', 'code': 'def plot_confusion_matrix(confusion_matrix, class_names):     """     Generates a confusion matrix plot from a sklearn confusion matrix object.      Parameters:     - confusion_matrix (array): Numpy array containing the confusion matrix information.     - class_names (list of str): List of class names corresponding to the labels.      Returns:     - fig (plotly.graph_objects.Figure): The Plotly figure object for the confusion matrix plot.     """     fig = ff.create_annotated_heatmap(         z=confusion_matrix,         x=class_names,         y=class_names,         colorscale=\'Blues\',         showscale=True     )     fig.update_layout(title=\'Confusion Matrix\', xaxis_title=\'Predicted Label\', yaxis_title=\'True Label\')     fig.update_traces(text=confusion_matrix.astype(str), texttemplate=\'%{text}\')     return fig'}, page_content='Generates a c

### Define the chain

In [ ]:
## Here we just define a simple prompt because after it we will refine it

prompt_handle_conver =  PromptTemplate.from_template("""
You are a conversational chatbot, designed to answer based mainly in the context you are provided to the user questions.
Context: {context}

User question: {question}
""")
semantic_chain = {'context':retriever,'question':RunnablePassthrough()} | prompt_handle_conver | llm



In [ ]:
semantic_chain.invoke("How to generate a confusion matrix with plotly?")

AIMessage(content='To generate a confusion matrix with Plotly, you can use the provided function `plot_confusion_matrix` from the `machine_learning_evaluation_plots.py` file. This function takes in a numpy array containing the confusion matrix information and a list of class names corresponding to the labels. It then generates a Plotly figure object for the confusion matrix plot. You can call this function with your confusion matrix array and class names to visualize the confusion matrix using Plotly.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 675, 'total_tokens': 766}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e47669dd-c87a-414a-82b8-ae4bdb09e210-0', usage_metadata={'input_tokens': 675, 'output_tokens': 91, 'total_tokens': 766})

<a id="6"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color: white; font-size:120%; text-align:left;padding:3.0px; background: maroon; border-bottom: 8px solid black" > Adding Memory<br><div>

As the final objective of our chain is to handle a conversation (Chatbot), we will add a memory component to the chain. We will show how to integrate this memory (a way of doing it), in both our two cases. This part will also cover the query rewritting functionality, based in the method that we want to apply

## Semantic search Chain 

To add memory to this step we will just add another call to an LLM with all the conversation provided by the RunnableWithMessageHistory wrapper. This call will have the objective to summarise the latests 3 (customizable) messages, to create a final message with full context and also already oriented for the purpose of querying the vector database.

In [ ]:

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


prompt_handle_conver =  PromptTemplate.from_template("""
You are a conversational chatbot, designed to answer based mainly in the context you are provided to the user questions.
Context: {context}

User question: {question}
""")

prompt_summarise_conver =  PromptTemplate.from_template("""
You are going to be provided with several consecutive messages of a user from a conversation.
Your task is to contextualize the latest message with anything relevant from the latest if it is necessary.
Also try to condense the information (without removing anything relevant) so the input is more usable as a query for a vector database.
                                                        
History of messages: {chat_history}

""")




First we handle the history summarisation:

In [ ]:
def select_last_n_messages(chat_history,n=3):
    print(chat_history)
    if len(chat_history) <3:
        return chat_history
    else:
        print("Shortering chat messages")
        print(chat_history)
        print(chat_history[-3:])
        return chat_history[-3:]
    

with_message_history = RunnableWithMessageHistory(
    {'chat_history': itemgetter('chat_history') | RunnableLambda(select_last_n_messages), "input": itemgetter('input') | RunnablePassthrough()  } | prompt_summarise_conver | llm,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [ ]:
with_message_history.invoke({ "input": "What is plotly?"},
    config={"configurable": {"session_id": "abc123"}})

[]


AIMessage(content='No history of messages provided.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 79, 'total_tokens': 85}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a5523a95-4878-46c1-a032-5d47a28949cd-0', usage_metadata={'input_tokens': 79, 'output_tokens': 6, 'total_tokens': 85})

In [ ]:
with_message_history.invoke({ "input": "And how to create a confusion matrix with it?"},
    config={"configurable": {"session_id": "abc123"}})

[HumanMessage(content='What is plotly?'), AIMessage(content='No history of messages provided.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 79, 'total_tokens': 85}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a5523a95-4878-46c1-a032-5d47a28949cd-0', usage_metadata={'input_tokens': 79, 'output_tokens': 6, 'total_tokens': 85})]


AIMessage(content='User is asking about what plotly is.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 225, 'total_tokens': 234}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ca16c2ed-370e-48d8-b45f-2eb5c82b9539-0', usage_metadata={'input_tokens': 225, 'output_tokens': 9, 'total_tokens': 234})

In [ ]:
with_message_history.invoke({ "input": "What?"},
    config={"configurable": {"session_id": "abc123"}})

[HumanMessage(content='What is plotly?'), AIMessage(content='No history of messages provided.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 79, 'total_tokens': 85}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a5523a95-4878-46c1-a032-5d47a28949cd-0', usage_metadata={'input_tokens': 79, 'output_tokens': 6, 'total_tokens': 85}), HumanMessage(content='And how to create a confusion matrix with it?'), AIMessage(content='User is asking about what plotly is.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 225, 'total_tokens': 234}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ca16c2ed-370e-48d8-b45f-2eb5c82b9539-0', usage_metadata={'input_tokens': 225, 'output_tokens': 9, 'total_tokens': 234})]
Shortering chat messages


AIMessage(content='User is asking about how to create a confusion matrix using Plotly.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 370, 'total_tokens': 384}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e7e94c1d-5575-46db-b5e6-1fa911ed0c47-0', usage_metadata={'input_tokens': 370, 'output_tokens': 14, 'total_tokens': 384})

As we have just seen, this does not make sense, because the variable under 'chat_history' provided by the wrapper is not considering the current message. So if we want to summarise the information so it helps contextualize the current message what we will need is to use both historic and new messages.

Also we are going to keep just the previous human messages, because our only objective as we said, is improve both the query to the database.

In [ ]:
## Reset the memory store:

store = {}
## Modify the function so we just get human messages
def select_last_n_messages(chat_history,n=3):

    # Extract content from the last 3 HumanMessages
    human_contents = [msg.content for msg in chat_history if isinstance(msg, HumanMessage)]
    # print(chat_history)
    if len(human_contents) <3:
        return "/n".join(human_contents)
    else:
        print("Shortering chat messages")
        print(human_contents)
        print(human_contents[-3:])
        return "/n".join(human_contents[-3:])
    
# Modify the prompt to summarise both history and current message together:
prompt_summarise_conver =  PromptTemplate.from_template("""
You are going to be provided with several consecutive messages of a user from a conversation, together with the current one.
Your task is to contextualize the current message with anything crutial from the oldest if it is necessary, so it can be understood alone. 
You should not change the format of the message and just create a final Input/question from the Current question, not older questions should be added.

Also try to condense the information (without removing anything relevant) so the input is more usable as a query for a vector database.
                                                        
History of messages: {chat_history}
                                                        
Current message: {input_message}
                                                        
Final Message: 

""")
    

with_message_history = RunnableWithMessageHistory(
   {"chat_history": itemgetter('chat_history') | RunnableLambda(select_last_n_messages), "input_message": itemgetter('input'), "input": itemgetter('input') | RunnablePassthrough()  } | prompt_summarise_conver | llm,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [ ]:
output = with_message_history.invoke({ "input": "What is plotly?"},
    config={"configurable": {"session_id": "abc123"}})

In [ ]:
output.content

'What is plotly? (Context: User is asking for information about the software or tool called Plotly.)'

As we can see the result is also containing text so the query to the vector database is optimized. This obviously could be improved. Also it would be better to use a query for the database and other for the LLM as user input. But right now, and considering that the queries would be pretty simple, we will keep the same.

In [ ]:
output = with_message_history.invoke({ "input": "How to create a confusion matrix with it?"},
    config={"configurable": {"session_id": "abc123"}})

In [ ]:
output.content

'How to create a confusion matrix with Plotly?'

Finally we add our retriever after this newly generated query

In [ ]:

with_message_history = RunnableWithMessageHistory(
   {"chat_history": itemgetter('chat_history') | RunnableLambda(select_last_n_messages), "input_message": itemgetter('input'), "input": itemgetter('input') | RunnablePassthrough()  } | prompt_summarise_conver | llm | StrOutputParser() | retriever,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [ ]:
with_message_history.invoke({ "input": "How to create a confusion matrix with it?"},
    config={"configurable": {"session_id": "abc123"}})

[Document(metadata={'file_path': 'visualization\\plotly\\machine_learning_evaluation_plots.py', 'name': 'plot_confusion_matrix', 'code': 'def plot_confusion_matrix(confusion_matrix, class_names):     """     Generates a confusion matrix plot from a sklearn confusion matrix object.      Parameters:     - confusion_matrix (array): Numpy array containing the confusion matrix information.     - class_names (list of str): List of class names corresponding to the labels.      Returns:     - fig (plotly.graph_objects.Figure): The Plotly figure object for the confusion matrix plot.     """     fig = ff.create_annotated_heatmap(         z=confusion_matrix,         x=class_names,         y=class_names,         colorscale=\'Blues\',         showscale=True     )     fig.update_layout(title=\'Confusion Matrix\', xaxis_title=\'Predicted Label\', yaxis_title=\'True Label\')     fig.update_traces(text=confusion_matrix.astype(str), texttemplate=\'%{text}\')     return fig'}, page_content='Generates a c

## Generate a response from this information

Finally, after condensing the latest messages, and rewritting (if necessary) the query, we have our information retrieved from the Vector Database. Now we just need to add the Conversational LLM.

In [ ]:

prompt_handle_conver =  PromptTemplate.from_template("""
You are a conversational chatbot, designed to answer based mainly in the context you are provided to the user questions.
Questions are most likely related to coding doubts. You should provide code examples whenever possible.
Context: {context}

User question: {input}
""")


with_message_history = RunnableWithMessageHistory(
   {"context": {"chat_history": itemgetter('chat_history') | RunnableLambda(select_last_n_messages), "input_message": itemgetter('input')}| prompt_summarise_conver | llm | StrOutputParser() | retriever , "input": RunnablePassthrough() }  | prompt_handle_conver | llm ,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)


output = with_message_history.invoke({ "input": "What is plotly?"},
    config={"configurable": {"session_id": "abc123"}})

output = with_message_history.invoke({ "input": "How to create a confusion matrix with it?"},
    config={"configurable": {"session_id": "abc123"}})

Shortering chat messages
['What is plotly?', 'How to create a confusion matrix with it?', 'How to create a confusion matrix with it?']
['What is plotly?', 'How to create a confusion matrix with it?', 'How to create a confusion matrix with it?']
Shortering chat messages
['What is plotly?', 'How to create a confusion matrix with it?', 'How to create a confusion matrix with it?', 'What is plotly?']
['How to create a confusion matrix with it?', 'How to create a confusion matrix with it?', 'What is plotly?']


In [ ]:
output.content

'To create a confusion matrix plot using Plotly, you can use the provided function `plot_confusion_matrix` defined in the code snippet. Here\'s how you can do it:\n\n```python\ndef plot_confusion_matrix(confusion_matrix, class_names):\n    """\n    Generates a confusion matrix plot from a sklearn confusion matrix object.\n    \n    Parameters:\n    - confusion_matrix (array): Numpy array containing the confusion matrix information.\n    - class_names (list of str): List of class names corresponding to the labels.\n    \n    Returns:\n    - fig (plotly.graph_objects.Figure): The Plotly figure object for the confusion matrix plot.\n    """\n    fig = ff.create_annotated_heatmap(\n        z=confusion_matrix,\n        x=class_names,\n        y=class_names,\n        colorscale=\'Blues\',\n        showscale=True\n    )\n    fig.update_layout(title=\'Confusion Matrix\', xaxis_title=\'Predicted Label\', yaxis_title=\'True Label\')\n    fig.update_traces(text=confusion_matrix.astype(str), textt

<a id="7"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color: white; font-size:120%; text-align:left;padding:3.0px; background: maroon; border-bottom: 8px solid black" > Final Chain<br><div>

Now that we have showed how to combine the memory with the retriever to generate a full chain, we will add in our retrieving chain also the Graph retriever

In [ ]:
langchain.debug = True
graph_retriever_chain =  cypher_gen_prompt | llm | RunnableLambda(run_cypher_query)

# In this case we also add the rephrasing/summarising from the history:
summarisation_chain =  {"chat_history": itemgetter('chat_history') | RunnableLambda(select_last_n_messages), "input_message": itemgetter('input')}| prompt_summarise_conver | llm | StrOutputParser()

# Therefore the final chain will by:

def context_unifier(full_context):
    print("Graph context--->",full_context['graph_context'])
    print("Vector context--->",full_context['vector_context'])
    unified_context = full_context['graph_context'] + full_context['vector_context'] 
    return unified_context

def get_schema(summarisation):
    return graph.get_schema

final_chain =  {'context' :summarisation_chain | {'graph_context': {'question': RunnablePassthrough(), 'schema': RunnableLambda(get_schema)} | graph_retriever_chain , 'vector_context':retriever} | RunnableLambda(context_unifier), 'input': itemgetter("input")} | prompt_handle_conver | llm
final_chain_history = RunnableWithMessageHistory(
   final_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)


output = final_chain_history.invoke({ "input": "What is plotly?"},
    config={"configurable": {"session_id": "abc123"}})

output = final_chain_history.invoke({ "input": "How to create a confusion matrix with it?"},
    config={"configurable": {"session_id": "abc123"}})

[chain/start] [chain:RunnableWithMessageHistory] Entering Chain run with input:
{
  "input": "What is plotly?"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history] Entering Chain run with input:
{
  "input": "What is plotly?"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history>] Entering Chain run with input:
{
  "input": "What is plotly?"
}
[chain/start] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history> > chain:load_history] Entering Chain run with input:
{
  "input": "What is plotly?"
}
[chain/end] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history> > chain:load_history] s] Exiting Chain run with output:
[outputs]
[chain/end] [chain:RunnableWithMessageHistory > chain:insert_history > chain:RunnableParallel<chat_history>] s] Exiting Chain run with output:
[outputs]
[chain/end] [chain:RunnableWithMessageHistory > chain

In [ ]:
output

AIMessage(content='To create a confusion matrix plot using Plotly, you can use the provided function `plot_confusion_matrix` defined in the code snippet. Here\'s how you can do it:\n\n```python\ndef plot_confusion_matrix(confusion_matrix, class_names):\n    """\n    Generates a confusion matrix plot from a sklearn confusion matrix object.\n    \n    Parameters:\n    - confusion_matrix (array): Numpy array containing the confusion matrix information.\n    - class_names (list of str): List of class names corresponding to the labels.\n    \n    Returns:\n    - fig (plotly.graph_objects.Figure): The Plotly figure object for the confusion matrix plot.\n    """\n    fig = ff.create_annotated_heatmap(\n        z=confusion_matrix,\n        x=class_names,\n        y=class_names,\n        colorscale=\'Blues\',\n        showscale=True\n    )\n    fig.update_layout(title=\'Confusion Matrix\', xaxis_title=\'Predicted Label\', yaxis_title=\'True Label\')\n    fig.update_traces(text=confusion_matrix.

<a id="8"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color: white; font-size:120%; text-align:left;padding:3.0px; background: maroon; border-bottom: 8px solid black" > Next Steps<br><div>

This is the initial development for the project ['Chat with your code'](https://github.com/hectorrrr/chat_with_your_code), but there are many potential improvements which we will address over a period of time. Here we list some of them:

* Custom query 'Rewritting' for both Graph and Vector search
* Test a 'Sequential' retriever instead of parallel, to not get duplicated data if unnecessary.
* Create evaluation metrics to optimize the flow